In [1]:
from darkflow.net.build import TFNet
import cv2
import numpy as np

#---------------------------------------------------------------------------------------------------------------------------------------------

options = {"pbLoad": "tiny-yolo-hand.pb", "threshold": 0.1, "metaLoad": "tiny-yolo-hand.meta", "gpu": 1.0} 
tfnet = TFNet(options)
global Ready_frame, Ready_flag
Ready_frame = [200,280,280,360]
Ready_flag = False

#---------------------------------------------------------------------------------------------------------------------------------------------
def boxing(original_img, predictions):
    global Ready_frame, Ready_flag
    newImage = np.copy(original_img)
    top, left, bottom, right = Ready_frame
    
    if not Ready_flag:
        newImage = cv2.rectangle(newImage, (left, top), (right, bottom), (0,0,255), 3)
        newImage = cv2.putText(newImage, 'Not ready', (left, top-5), cv2.FONT_HERSHEY_COMPLEX_SMALL , 0.8, (0, 230, 0), 1, cv2.LINE_AA)
    else :
        newImage = cv2.rectangle(newImage, (left, top), (right, bottom), (0,255,0), 3)
        newImage = cv2.putText(newImage, 'Ready', (left, top-5), cv2.FONT_HERSHEY_COMPLEX_SMALL , 0.8, (0, 230, 0), 1, cv2.LINE_AA)
        
    for result in predictions:
        top_x = result['topleft']['x']
        top_y = result['topleft']['y']

        btm_x = result['bottomright']['x']
        btm_y = result['bottomright']['y']
        
        center_x = (top_x+btm_x)/2
        center_y = (top_y+btm_y)/2

        confidence = result['confidence']
        label = result['label'] + " " + str(round(confidence, 3))
        
        if Ready_flag:
            if confidence > 0.1:
                newImage = cv2.rectangle(newImage, (top_x, top_y), (btm_x, btm_y), (255,0,0), 3)
                newImage = cv2.putText(newImage, label, (top_x, top_y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL , 0.8, (0, 230, 0), 1, cv2.LINE_AA)
                if center_y<top: 
                    newImage = cv2.putText(newImage, 'Top', (20, 20), cv2.FONT_HERSHEY_COMPLEX_SMALL , 0.8, (0, 230, 0), 1, cv2.LINE_AA)
                    Ready_flag = False
                elif center_y>bottom: 
                    newImage = cv2.putText(newImage, 'Bottom', (20, 20), cv2.FONT_HERSHEY_COMPLEX_SMALL , 0.8, (0, 230, 0), 1, cv2.LINE_AA)
                    Ready_flag = False
                if center_x<left:
                    newImage = cv2.putText(newImage, 'Left', (20, 50), cv2.FONT_HERSHEY_COMPLEX_SMALL , 0.8, (0, 230, 0), 1, cv2.LINE_AA)
                    Ready_flag = False
                elif center_x>right: 
                    newImage = cv2.putText(newImage, 'Right', (20, 50), cv2.FONT_HERSHEY_COMPLEX_SMALL , 0.8, (0, 230, 0), 1, cv2.LINE_AA)
                    Ready_flag = False
        else:
            if confidence > 0.1:
                newImage = cv2.rectangle(newImage, (top_x, top_y), (btm_x, btm_y), (255,0,0), 3)
                newImage = cv2.putText(newImage, label, (top_x, top_y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL , 0.8, (0, 230, 0), 1, cv2.LINE_AA)
                if center_x > left and center_x < right and center_y > top and center_y < bottom:
                    Ready_flag = True
                
            
            
            
    return newImage
#---------------------------------------------------------------------------------------------------------------------------------------------
cap = cv2.VideoCapture(0) # 0 represents the in-built camera (It can also be changed to the name of a video file or other camera)
print("Start detecting")
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read() #ret is boolean value (True for frame reading correctly otherwise False)
    frame = cv2.flip(frame, 1)
    result = tfnet.return_predict(frame)
    cv2.imshow('frame',boxing(frame,result)) 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()





Loading from .pb and .meta
GPU mode with 1.0 usage
Start detecting
